# **Part 2: Agentic Workflows**

![LangChain Logo](figs/AI_Agent.png)

### Imports & Setting up Gemini's API key

In [ ]:
import os, getpass
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

MODEL_NAME = "gemini-2.0-flash"  # or, feel free to use any LLM here
llm = ChatGoogleGenerativeAI(model=MODEL_NAME, temperature=0)

# Task 1: Function and Tool Calling with Agents

In this task, you will explore how modern LLMs can interact with external tools and functions to perform real-world tasks beyond simple text generation.  
You will learn how to **connect an LLM to APIs, databases, and system functions**, enabling it to perform practical operations such as searching the web, managing tasks, and controlling a simulated household device.



#### Overview: What Is Function Calling in LLMs?

**Function calling** allows an LLM to call specific functions or APIs during a conversation based on the user’s request.  
Rather than relying only on natural language generation, the model determines *when and how* to invoke a tool, API, or system function to gather data, perform computations, or change system states.

This mechanism bridges the gap between **language understanding** and **real-world action**.  
It makes the LLM an **active reasoning agent** that can access structured tools or APIs when it detects that such calls are necessary.



#### Why Web Search Tools Matter: Tavily Search API

For this task, we will use the **Tavily Search API** — a web search API designed specifically for LLM applications.  
You can set up Tavily by visiting the official website and generating an API key.

---

## Task Scenario: Household Assistant Application

You will design a small **Household Assistant Application** where an LLM uses function calling to manage everyday tasks.  
The system will include a series of tool functions that the model can invoke based on user prompts.  

The LLM will decide which tool to use and automatically format the function call as defined in its configuration.

---

## Functions to Implement

### 1. Web Search Function

This function will use the **Tavily Search API** to perform web searches when the LLM identifies that external information is needed.  
You should configure your Tavily API key and handle the returned search results.

### 2. To-Do List

Your assistant will maintain a to-do list stored as text embeddings in a vector store (e.g., Chroma).

You'll need a tool for adding to a To-Do List, and a tool to query from that To-Do List

### 3. Get Current Time

A tool to get the current system time

### 4. Toggle Light On/Off

This function simulates turning a light on or off in your home. The current state of the light needs to be stored. 
You also need two tools for this. One to toggle it on (if it's off) or off (if it's on), and the other to check and return the current state.

You can visit the official website and see how to set up the Tavily Search API:

[Tavily Resource](https://docs.tavily.com/documentation/integrations/langchain)

In [ ]:
from langchain_core.tools import tool

#TODO: Set up the Web Search tool
@tool
def web_search(query: str) -> str:
    pass

In [ ]:
#TODO: Write code for the other tools

In [ ]:
#TODO: Create an Agent Extractor

#### Evaluating Your Agent's Function Calls

Next, you will assess your agent's performance using a set of predefined queries.  
These queries are provided in a dictionary where:

- **Key:** The user query  
- **Value:** A list of the expected function calls that the agent should invoke  

Use this information to calculate the **Average Function Call Accuracy (FCA)** for your agent, which measures how accurately your agent chooses the correct functions across all queries.

You can access this dictionary from the `datasets/task2/queries_and_expected_fcs.json` file

#### **Reflective Question:** Did multiple function calls by the LLM affect the performance of the overall result?

---

# Task 2: Multi-Agent Debate System

In this task, you will build a **multi-agent system** that "debates" and "refines" a creative idea to reach a consensus. The system simulates a movie studio pitch meeting using three distinct AI agents:

- **Cora (The Creative):** The passionate, idea-generating screenwriter.  
- **Barnaby (The Numbers Guy):** The pragmatic, budget-focused studio executive. 
- **Vera (The Tech & Effects Specialist):** Focuses on technical feasibility, special effects, and visual/audio production planning.
- **Ames (The Producer):** The moderator and final decision-maker.

The goal is to take a simple, one-line movie idea and develop it into a finalized movie script which is **high-quality, greenlit and ready for production**.

### Scenario

You are an AI engineer at **Skynet Pictures**. Your job is to automate the initial pitch-development process. A user will provide a simple idea (e.g., `"zombies but in space"`), and your team of agents (Cora, Barnaby, Vera) must:

1. Debate the concept, considering both creative and financial aspects.
2. Refine the idea iteratively.
3. Produce a final script that Ames (The Producer) can approve for production.

### Tools to be used by each agent are mentioned as follows:

#### **Global Tools (Available to All Agents)**

| Tool Name                            | Purpose / Description                                      | Example Use Cases                                                                                      |
| ------------------------------------ | ---------------------------------------------------------- | ------------------------------------------------------------------------------------------------------ |
| `check_current_time()`               | Returns the current system time in a human-readable format | Reference time-sensitive ideas; schedule shoots; make deadline decisions                               |
| `web_search(query: str)`             | Searches the web for relevant information                 | Research story ideas, check market trends, fact-check feasibility                                      |
| `query_todo()`                       | Returns current to-do tasks                                | Track agreed-upon story or production tasks during debate                                              |
| `add_to_todo(task: str)`             | Adds a task to the shared to-do list                       | Propose action items like "refine scene" or "budget review"                                           |
| `get_weather(location: str)`         | Returns weather forecast                                   | Plan scenes or production dependent on weather                                                        |
| `calculate_budget(cost_items: dict)` | Returns total cost estimate                                | Quickly estimate total costs for proposed story elements                                              |

---

#### **Cora – Creative Screenwriter**

| Tool Name                                  | Purpose / Description                          | Example Use Cases                                            |
| ------------------------------------------ | ---------------------------------------------- | ------------------------------------------------------------ |
| `generate_plot_twist(base_plot: str)`      | Suggests plot twists or story enhancements     | "Add a surprising zombie twist in zero gravity"             |
| `suggest_character(name: str, role: str)`  | Creates a character profile                    | "Suggest the main protagonist for a sci-fi horror movie"    |
| `scene_visualizer(scene_description: str)` | Outputs a short visual description of a scene  | Helps communicate mood and setting to other agents          |
| `tone_analyzer(text: str)`                 | Determines tone/emotion of a scene or dialogue | Ensure story tone matches target audience expectations      |

---

#### **Barnaby – Studio Executive**

| Tool Name                                  | Purpose / Description                        | Example Use Cases                                      |
| ------------------------------------------ | -------------------------------------------- | ------------------------------------------------------ |
| `estimate_actor_cost(actor_name: str)`     | Returns estimated cost for a given actor     | Assess casting feasibility                             |
| `market_trend_check(genre: str)`           | Returns popularity metrics for a movie genre | Determine if "zombies in space" is marketable         |
| `schedule_production(days_needed: int)`    | Returns suggested shooting schedule          | Optimize production timeline based on proposed scenes  |

---

#### **Vera – Tech & Effects Specialist**

| Tool Name                                                   | Purpose / Description                                                                  | Example Use Cases                                                                            |
| ----------------------------------------------------------- | -------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------- |
| `evaluate_vfx_complexity(scene_description: str)`           | Estimates complexity and technical difficulty of visual effects                         | Determine feasibility of complex scenes like “zombies floating in zero gravity”             |
| `suggest_camera_angles(scene_description: str)`             | Suggests cinematic camera angles and shots for a scene                                  | Visualize key moments for storyboarding                                                     |
| `estimate_postproduction_time(scene_description: str)`      | Predicts how long postproduction will take                                             | Plan production schedules based on scene complexity                                        |
| `audio_effect_suggestion(scene_description: str)`           | Suggests sound effects or background audio                                             | Enhance immersion for horror or action sequences                                           |
| `simulate_scene_budget(vfx_cost: float, audio_cost: float)` | Combines VFX and audio costs to provide a rough scene budget                            | Assess if a scene fits within overall budget                                              |

---

#### **Ames – Producer / Moderator**

| Tool Name                                      | Purpose / Description                     | Example Use Cases                                                 |
| ---------------------------------------------- | ----------------------------------------- | ----------------------------------------------------------------- |
| `summarize_discussion(conversation: list)`     | Summarizes agent debate history           | Provides a concise summary of pros/cons for final decision        |
| `decision_matrix(options: list, scores: dict)` | Scores options based on multiple criteria | Greenlight the best concept considering creativity + budget       |
| `highlight_conflicts(conversation: list)`      | Detects conflicting opinions among agents | Mediate conflicts between Cora, Barnaby, and Vera                 |
| `final_pitch_generator(conversation: list)`    | Produces a polished, greenlit movie pitch | Synthesize debate into a coherent pitch ready for approval        |
| `validate_todo()`                              | Checks that all agreed tasks are feasible | Ensure all follow-up action items are realistic                  |

---

## Requirements of the Task

 **Note:** You would be graded on the quality of your prompts (both system and user messages), context engineering, overall code setup and the finalized movie script. You don't need to worry if your LLM does not use a certain set of tools. You'd still be graded on the functions that you initialized for those tools.

 For tools that do not require calling external APIs or Python functions, you should implement them using **LangChain chains**.  

 This means that instead of directly executing a function, you create a chain that encapsulates the logic or reasoning for that tool. For example, if you have a tool like `cinematic_location_cost(location: str)` to estimate location rental costs, you would:

 1. Define a chain that provides relevant context about the task or scenario.
 2. Pass the location as input to the chain.
 3. Return the output produced by the chain as the tool's result.
 
 Using chains in this way allows the agent to reason through the task and generate outputs dynamically, without relying on external code or APIs.

In [ ]:
# Your code here